## Debias Convex Algorithm

### Input:

- O: the observation
- Z: the treatment pattern
- $\lambda$: the parameter for nuclear norm

### Output:
The goal is to minimize
$$
\min_{M,\tau} \frac{1}{2p}\|P_{\Omega}(O-M-\tau Z)\|_{F}^2 + \lambda \| M\|_{*} 
$$

We do this by implementing an iterative algorithm:

- $M_0 = 0, \tau_0 = 0, t = 0$
- $t = t + 1$
- $M_t = \text{soft\_thresholding}(O-\tau Z, \lambda)$ 
- $\tau_{t} = <Z,O-M_t>/\|Z\|_{F}^2$
- Until $|\tau_{t} - \tau_{t-1}| < 1e-3$

## MC-NNM (Bayati etc. 2016 algorithm)

### View treatment as missing entries, then use convex optimization

### Input:
        
- O: the observation
- Ω: the set of no-treatment (Ω = 1 - Z)
- l: the parameter for nuclear norm minimization

### Output:

#### Without fixed effects

solve the optimization:
Let p be the observation probability: $ p =|\Omega| / \text{np.size}(O)$,
\begin{align}
\min_{M} \frac{1}{2p} \|P_{\Omega}(O-M)\|_{F}^2 + \lambda \|M\|_{*}
\end{align}

We do this by iteration (soft_impute)

- $M_0 = 0$
- $M_t = \text{soft\_thresholding}(Ω*O + (1-Ω)*M, \lambda * p)$

After convergence, obtaining $M$, Let
$$
\tau = <1-Ω, O-M> / \|1-Ω\|_{F}^2
$$

#### With fixed effects

solve the optimization
\begin{align}
\min_{M} \frac{1}{2p} \|P_{\Omega}(O - a1^{T} - 1b^{T} - M)\|_{F}^2 + \lambda \|M\|_{*}
\end{align}

We do this by:

- $M_0 = 0, a^0 = 0, b^0 = 0$
- fix $a^t, b^t$, solve $M_t = \text{soft\_thresholding}(Ω*(O-a^t1^{T}-1b^{tT}) + (1-Ω)*M, \lambda * p)$
- fix $M_t$, solve the following convex optimization
$$
\min_{a \in R^{n}, b \in R^{n}} \sum_{(i,j)\in \Omega} (O_{ij}-M_{t,ij} - a_{i} - b_{j})^2
$$

we solve the above by iteration again (one may change to solve a linear system by considering the first-order condition, not sure which one is quicker)
- fix $b$, 
$$ 
a_i = \frac{\sum_{j, (i,j) \in \Omega} O_{ij} - M_{t,ij} - b_{j}}{\sum_{j, (i, j) \in \Omega} 1}
$$
- fix $a$
$$
b_j = \frac{\sum_{i, (i,j) \in \Omega} O_{ij} - M_{t,ij} - a_{i}}{\sum_{i, (i, j) \in \Omega} 1}
$$


## Covariance PCA, Xiong and Pelger, 2019

Input the matrix $O \in R^{n\times T}$ and Ω

### Step 1: estimate loadings from covariance matrix

\begin{align}
A_{ij} = \frac{1}{\sum_{k, Ω_{i,k} = Ω_{j,k}=1} 1}\sum_{k, Ω_{i,k} = Ω_{j,k}=1} O_{i,k} \cdot O_{j,k}
\end{align}

U = $\sqrt{n}$ * First r eigenvectors of A, therefore $B \in R^{n\times r}$

### Step 2: regress the factors
Let $Y \in R^{T\times r}$.
\begin{align}
Y_{i,\cdot} = \frac{1}{\sum_{k, Ω_{k,i}=1} 1}\sum_{k, Ω_{k,i}=1} O_{k,i} U_{k,\cdot}  
\end{align}

$M = U \cdot Y^{T}$ is the estimator for $M^{*}$

### Step 3: estimate $\tau$
\begin{align}
\tau = \frac{1}{|Z|_0}<Z, O-M>.
\end{align}

## Robust Synthetic Control Shah etc 2017

## Treatment Pattern Generation

### Block Pattern


### i.i.d Pattern


### Two Segment Pattern

### Adaptive Treatment Pattern

#### Input: lowest_T, lasting_T, M

For each row i, if M(i,j) is the smallest among M(i, j-lowest_T:j) and no treatments on (i, j-lowest_T:j), then start the treatment on M(i,j+1) to M(i,j+lasting_T+1)

In [40]:
def adpative_treatment_pattern(lowest_T, lasting_T, M):
    Z = np.zeros_like(M)
    for i in range(Z.shape[0]):
        j = 0
        #print(i)
        while j < Z.shape[1]:
            flag = 0
            for k in range(1, lowest_T+1):
                if (j-k < 0 or Z[i, j-k]==1 or M[i,j] > M[i,j-k]):
                    flag = 1
                    break

            if (flag == 0):
                for k in range(1, lasting_T+1):
                    if (j+k < Z.shape[1]):
                        Z[i, j+k] = 1
                j += lasting_T + lowest_T
            else:
                j = j + 1
    return Z